In [1]:
import numpy as np
import pandas as pd
import datetime
import math
#from os import listdir
#from os.path import isfile, join

In [13]:
#from os import listdir
#from os.path import isfile, join
#pth='monthly/'
#files = [f for f in listdir(pth) if isfile(join(pth, f))]
#dam = pd.DataFrame()
#for file in files:
#    print(file)
#    fm = pd.read_csv('monthly/' + file)
#    dam=pd.concat([dam,fm])
    
#dam

In [26]:
dam = pd.read_csv('dam.csv')
dam

,hourlyid,market,tradedate,hour,price,volume
0,26345,1,2016-01-01,0,40.10,477.5
1,26346,1,2016-01-01,1,40.08,486.6
2,26347,1,2016-01-01,2,40.06,495.6
3,26348,1,2016-01-01,3,40.08,496.6
4,26349,1,2016-01-01,4,40.07,502.9
...,...,...,...,...,...,...
46723,73029,1,2021-04-29,4,39.97,126.1
46724,73037,1,2021-04-29,12,68.06,71.0
46725,73049,1,2021-04-30,0,44.04,59.6
46726,73057,1,2021-04-30,8,190.00,72.0


In [27]:
dam['date']=pd.to_datetime(dam['tradedate'],format="%Y/%m/%d")#"%d/%m/%Y")
dam=dam.set_index(pd.DatetimeIndex(dam['date']))
dam['year']=dam.index.year
dam['month']=dam.index.month
dam['day']=dam.index.day
dam['day_of_week']=dam.index.dayofweek
dam['month_no']=dam['year']+dam['month']/100

dam['hour_ref'] = dam.apply(lambda x: ('weekend_' + str(x['hour']) if x['day_of_week']>4 else 'weekday_' + str(x['hour'])),axis=1)
dam=dam.drop(['hourlyid','tradedate','date'],axis=1)
dam

,market,hour,price,volume,year,month,day,day_of_week,month_no,hour_ref
date,,,,,,,,,,
2016-01-01,1,0,40.10,477.5,2016,1,1,4,2016.01,weekday_0
2016-01-01,1,1,40.08,486.6,2016,1,1,4,2016.01,weekday_1
2016-01-01,1,2,40.06,495.6,2016,1,1,4,2016.01,weekday_2
2016-01-01,1,3,40.08,496.6,2016,1,1,4,2016.01,weekday_3
2016-01-01,1,4,40.07,502.9,2016,1,1,4,2016.01,weekday_4
...,...,...,...,...,...,...,...,...,...,...
2021-04-29,1,4,39.97,126.1,2021,4,29,3,2021.04,weekday_4
2021-04-29,1,12,68.06,71.0,2021,4,29,3,2021.04,weekday_12
2021-04-30,1,0,44.04,59.6,2021,4,30,4,2021.04,weekday_0


In [28]:
dam['value']=dam['price']*dam['volume']
daily=dam.groupby(['year','month','day']).sum().drop(['hour','day_of_week','month_no','market'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=dam[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=dam[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=dam[['year','month','day','price']].groupby(['year','month','day']).mean()

daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2021 4     26   2092.9  185605.239        88.683281     299.33      39.94   
           27   2048.2  168261.497        82.150912     299.09      39.96   
           28   2130.2  177543.993        83.346161     299.10      39.90   
           29   2134.0  178697.984        83.738512     296.82      39.79   
           30   2198.9  187057.475        85.068659     295.98      38.97   

                price_mean  
year month day              
2016 1     1     45.982500  
           2     54.114167  
           3     54.959167  
           4     80.164583  
           5     84.503333  
...                    ...  
2021 4     26   102.215000  
           27    94.468333  
           28   100.343333  
           29    98.742500  
           30    98.218750  

[1947 rows x 6 columns]

In [29]:
dam=dam.reset_index().set_index(['year','month','day'])
dam

date  market  hour   price  volume  day_of_week  \
year month day                                                         
2016 1     1   2016-01-01       1     0   40.10   477.5            4   
           1   2016-01-01       1     1   40.08   486.6            4   
           1   2016-01-01       1     2   40.06   495.6            4   
           1   2016-01-01       1     3   40.08   496.6            4   
           1   2016-01-01       1     4   40.07   502.9            4   
...                   ...     ...   ...     ...     ...          ...   
2021 4     29  2021-04-29       1     4   39.97   126.1            3   
           29  2021-04-29       1    12   68.06    71.0            3   
           30  2021-04-30       1     0   44.04    59.6            4   
           30  2021-04-30       1     8  190.00    72.0            4   
           30  2021-04-30       1    17   67.97   114.2            4   

                month_no    hour_ref      value  
year month day                                   
2016 1     1     2016.01   weekday_0  19147.750  
           1     2016.01   weekday_1  19502.928  
           1     2016.01   weekday_2  19853.736  
           1     2016.01   weekday_3  19903.728  
           1     2016.01   weekday_4  20151.203  
...                  ...         ...        ...  
2021 4     29    2021.04   weekday_4   5040.217  
           29    2021.04  weekday_12   4832.260  
           30    2021.04   weekday_0   2624.784  
           30    2021.04   weekday_8  13680.000  
           30    2021.04  weekday_17   7762.174  

[46728 rows x 9 columns]

In [30]:


daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2021 4     26   2092.9  185605.239        88.683281     299.33      39.94   
           27   2048.2  168261.497        82.150912     299.09      39.96   
           28   2130.2  177543.993        83.346161     299.10      39.90   
           29   2134.0  178697.984        83.738512     296.82      39.79   
           30   2198.9  187057.475        85.068659     295.98      38.97   

                price_mean  
year month day              
2016 1     1     45.982500  
           2     54.114167  
           3     54.959167  
           4     80.164583  
           5     84.503333  
...                    ...  
2021 4     26   102.215000  
           27    94.468333  
           28   100.343333  
           29    98.742500  
           30    98.218750  

[1947 rows x 6 columns]

In [31]:

dam['price_min']=daily['price_min']
dam['price_max']=daily['price_max']
dam['price_mean']=daily['price_mean']

dam['cat']='standard'
dam.loc[(dam['day_of_week']<5) & (dam['price'] > dam['price_max']-(dam['price_mean'] * 0.2)),'cat']='peak'
dam.loc[dam['price'] < dam['price_min']+(dam['price_mean'] * 0.2),'cat']='off'

dam.loc[dam['price_max'] - dam['price_min']<30,'cat']='off'
dam

date  market  hour   price  volume  day_of_week  \
year month day                                                         
2016 1     1   2016-01-01       1     0   40.10   477.5            4   
           1   2016-01-01       1     1   40.08   486.6            4   
           1   2016-01-01       1     2   40.06   495.6            4   
           1   2016-01-01       1     3   40.08   496.6            4   
           1   2016-01-01       1     4   40.07   502.9            4   
...                   ...     ...   ...     ...     ...          ...   
2021 4     29  2021-04-29       1     4   39.97   126.1            3   
           29  2021-04-29       1    12   68.06    71.0            3   
           30  2021-04-30       1     0   44.04    59.6            4   
           30  2021-04-30       1     8  190.00    72.0            4   
           30  2021-04-30       1    17   67.97   114.2            4   

                month_no    hour_ref      value  price_min  price_max  \
year month day                                                          
2016 1     1     2016.01   weekday_0  19147.750      35.09      60.08   
           1     2016.01   weekday_1  19502.928      35.09      60.08   
           1     2016.01   weekday_2  19853.736      35.09      60.08   
           1     2016.01   weekday_3  19903.728      35.09      60.08   
           1     2016.01   weekday_4  20151.203      35.09      60.08   
...                  ...         ...        ...        ...        ...   
2021 4     29    2021.04   weekday_4   5040.217      39.79     296.82   
           29    2021.04  weekday_12   4832.260      39.79     296.82   
           30    2021.04   weekday_0   2624.784      38.97     295.98   
           30    2021.04   weekday_8  13680.000      38.97     295.98   
           30    2021.04  weekday_17   7762.174      38.97     295.98   

                price_mean       cat  
year month day                        
2016 1     1      45.98250       off  
           1      45.98250       off  
           1      45.98250       off  
           1      45.98250       off  
           1      45.98250       off  
...                    ...       ...  
2021 4     29     98.74250       off  
           29     98.74250  standard  
           30     98.21875       off  
           30     98.21875  standard  
           30     98.21875  standard  

[46728 rows x 13 columns]

In [35]:
daily['off_price_mean']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['peak_price_mean']=dam.loc[dam['cat']=='peak'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['peak_hour_count']=dam.loc[dam['cat']=='peak'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['peak_volume']=dam.loc[dam['cat']=='peak'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()


daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2021 4     26   2092.9  185605.239        88.683281     299.33      39.94   
           27   2048.2  168261.497        82.150912     299.09      39.96   
           28   2130.2  177543.993        83.346161     299.10      39.90   
           29   2134.0  178697.984        83.738512     296.82      39.79   
           30   2198.9  187057.475        85.068659     295.98      38.97   

                price_mean  off_price_mean  standard_price_mean  \
year month day                                                    
2016 1     1     45.982500       45.982500                  NaN   
           2     54.114167       45.511333            68.452222   
           3     54.959167       48.152222            75.380000   
           4     80.164583       47.992222            80.720833   
           5     84.503333       43.756667            97.805385   
...                    ...             ...                  ...   
2021 4     26   102.215000       42.868889           112.993077   
           27    94.468333       42.197778            99.176154   
           28   100.343333       42.605556           109.740000   
           29    98.742500       42.532222           107.186154   
           30    98.218750       42.047778           106.682308   

                peak_price_mean  off_hour_count  standard_hour_count  \
year month day                                                         
2016 1     1                NaN              24                  NaN   
           2                NaN              15                  9.0   
           3                NaN              18                  6.0   
           4         174.456667               9                 12.0   
           5         181.400000               9                 13.0   
...                         ...             ...                  ...   
2021 4     26        299.215000               9                 13.0   
           27        299.085000               9                 13.0   
           28        299.085000               9                 13.0   
           29        296.805000               9                 13.0   
           30        295.975000               9                 13.0   

                peak_hour_count  off_volume  standard_volume  peak_volume  
year month day                                                             
2016 1     1                NaN      7873.8              NaN          NaN  
           2                NaN      5994.5           1325.0          NaN  
           3                NaN      6607.3            591.4          NaN  
           4                3.0      4028.0            429.0          0.0  
           5                2.0      3380.0            370.4          0.0  
...                         ...         ...              ...          ...  
2021 4     26               2.0       789.4           1209.4         94.1  
           27               2.0       947.8            990.7        109.7  
           28               2.0      1095.8            924.4        110.0  
           29               2.0      1100.0            914.0        120.0  
           30               2.0       912.7           1166.2        120.0  

[1947 rows x 15 columns]

In [36]:
daily.loc[2019,2,1]

volume                   6164.300000
value                  282329.556000
price_weight_av            45.800749
price_max                  92.000000
price_min                  25.050000
price_mean                 57.436667
off_price_mean             27.187500
standard_price_mean        62.882000
peak_price_mean            88.693333
off_hour_count              8.000000
standard_hour_count        10.000000
peak_hour_count             6.000000
off_volume               3472.000000
standard_volume          1928.300000
peak_volume               764.000000
Name: (2019, 2, 1), dtype: float64

In [37]:
daily.to_csv('dam_daily.csv')